## 1. 라이브러리 설치

In [80]:
# pandas 설치
import pandas as pd

In [81]:
# 환경변수 
from dotenv import load_dotenv
import os

load_dotenv()

file_path = os.environ.get('RAW_DATA_PATH')

## 2. 데이터 수집
- 데이터 수집 후 필요 속성 null이 없는 데이터셋 구축

### 공공 데이터 포털
- https://www.data.go.kr/data/15100070/standard.do?recommendDataYn=Y

In [82]:
# csv파일 Load
try:
    df = pd.read_csv(file_path + '원본 음식 데이터/전국통합식품영양성분정보 표준데이터.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path + '원본 음식 데이터/전국통합식품영양성분정보 표준데이터.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [83]:
# '데이터구분명'의 값이 '음식'이고, 주어진 컬럼들에 null값이 없는 데이터 추출
required_columns = ['에너지(kcal)', '영양성분함량기준량', '단백질(g)', '지방(g)', '탄수화물(g)', '당류(g)', '나트륨(mg)', '식이섬유(g)']
filtered_df = df[(df['데이터구분명'] == '가공식품') & (df[required_columns].notnull().all(axis=1))]

In [84]:
# 총 인스턴스 개수 파악
total_instances = len(filtered_df)

In [85]:
# 총 인스턴스 개수 : 1175개
print(f"\n총 인스턴스 개수: {total_instances}개")


총 인스턴스 개수: 1175개


In [7]:
# # 필터링된 데이터를 CSV 파일로 저장
# # utf-8-sig로 인코딩하여 한글 깨짐 방지
# filtered_df.to_csv('01_filtered_data_공공데이터.csv', index=False, encoding='utf-8-sig')

### 식품의약품안전처
- https://various.foodsafetykorea.go.kr/nutrient/

**가공식품**

In [86]:
# Excel 파일 Load
df = pd.read_excel(file_path + '원본 음식 데이터/식품의약품안전처_가공식품.xlsx')

In [87]:
# 주어진 컬럼들에 null값이 없는 데이터 추출
required_columns = ['에너지\n(kcal)', '영양성분기준용량', '단백질\n(g)', '지방\n(g)', '탄수화물\n(g)', '당류\n(g)', '나트륨\n(mg)', '식이섬유\n(g)']
filtered_df = df[df[required_columns].notnull().all(axis=1)]

In [88]:
# 총 인스턴스 개수 파악
total_instances = len(filtered_df)

In [89]:
print(f"\n총 인스턴스 개수: {total_instances}개")


총 인스턴스 개수: 722개


In [90]:
# # 필터링된 데이터를 CSV 파일로 저장
# # utf-8-sig로 인코딩하여 한글 깨짐 방지
# filtered_df.to_csv('02_filtered_data_식품의약처_가공.csv', index=False, encoding='utf-8-sig')

**음식**

In [91]:
# Excel 파일 Load
df = pd.read_excel(file_path + '원본 음식 데이터/식품영양성분_음식.xlsx')

/Users/kyeong6/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [92]:
# 주어진 컬럼들에 null값이 없는 데이터 추출
required_columns = ['에너지(㎉)', '1회제공량', '단백질(g)', '지방(g)', '탄수화물(g)', '총당류(g)', '나트륨(㎎)', '총 식이섬유(g)']
filtered_df = df[df[required_columns].notnull().all(axis=1)]

In [93]:
# 총 인스턴스 개수 파악
total_instances = len(filtered_df)

In [94]:
print(f"\n총 인스턴스 개수: {total_instances}개")


총 인스턴스 개수: 7683개


In [16]:
# # 필터링된 데이터를 CSV 파일로 저장
# # utf-8-sig로 인코딩하여 한글 깨짐 방지
# filtered_df.to_csv('03_filtered_data_식품영양성분_음식.csv', index=False, encoding='utf-8-sig')

## 데이터 전처리
- DB에 넣기위한 데이터 가공 수행

### 01.Excel을 이용하여 필요 속성의 이름 동일하게 맞추기
    1. 식품명
    2. 1회제공량
    3. 에너지(kcal)
    4. 탄수화물(g)
    5. 단백질(g)
    6. 지방(g)
    7. 당류(g)
    8. 식이섬유(g)
    9. 나트륨(mg)

### 02.속성명을 동일하게 하는 과정에서 데이터셋 간의 차이 해결
- 사전에 전처리 필요한 데이터셋끼리 구조 맞추기
- “영양성분함량기준량”을 “1회섭취참고량”의 비율로 영양성분들의 값 변경
- 최종적으로 “1회 섭취참고량”을 “1회제공량” 속성명으로 변경한 후 “영양성분함량기준량” 속성 제거
- 03_Dataset 01,02_Dataset에 맞추기(구조)

In [102]:
# 전처리 함수 정의
def adjust_nutrient_values(df):
    # 영양성분함량기준량과 1회섭취참고량 속성에 존재하는 단위 'g' 또는 'ml' 제거하고 숫자형으로 변환
    df['영양성분함량기준량'] = df['영양성분함량기준량'].str.replace('g|ml', '', regex=True).replace('-', 0).astype(float)
    df['1회제공량'] = df['1회섭취참고량'].str.replace('g|ml', '', regex=True).replace('-', 0).astype(float)
    
    # "영양성분함량기준량"을 "1회제공량"의 비율로 숫자형 컬럼 값 변경 (소수점 2자리까지 반올림)
    ratio = df['1회제공량'] / df['영양성분함량기준량']
    for col in df.columns:
        if df[col].dtype == 'float64' or df[col].dtype == 'int64':
            df[col] = round(df[col] * ratio, 2)
            df[col].replace('-', 0, inplace=True)
    
    # 기존에 '1회섭취참고량' 컬럼 삭제
    df.drop(columns=['1회섭취참고량'], inplace=True)
    
    # 데이터셋 구조를 맞추기 위한 컬럼명 변경
    df.rename(columns={'영양성분함량기준량': '1회제공량'}, inplace=True)
    
    return df

In [103]:
# 전처리 필요한 csv 파일 업로드
df_01 = pd.read_csv(file_path + '1단계 가공 데이터/01_filtered_data_공공데이터.csv')
df_02 = pd.read_csv(file_path + '1단계 가공 데이터/02_filtered_data_식품의약처_가공.csv')

In [104]:
# 함수 실행
adjust_nutrient_values(df_01)

/var/folders/gp/txd8895x3vbc94rnxd7b3gjw0000gn/T/ipykernel_5219/2339764252.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace('-', 0, inplace=True)


,식품명,1회제공량,에너지(kcal),수분(g),단백질(g),지방(g),탄수화물(g),당류(g),식이섬유(g),칼슘(mg),...,베타카로틴(μg),티아민(mg),리보플라빈(mg),니아신(mg),비타민 C(mg),비타민 D(μg),콜레스테롤(mg),포화지방산(g),트랜스지방산(g),1회제공량
0,시래기나물무침,50.0,57.0,39.35,2.16,3.64,3.87,0.27,3.35,93.0,...,70.0,0.14,0.02,0.40,0.08,0.0,0.00,0.45,0.01,25.0
1,씀바귀나물무침,90.0,87.3,68.94,3.48,1.56,14.91,1.15,4.68,41.4,...,45.0,0.00,0.09,2.31,1.28,0.0,0.00,0.32,0.07,81.0
2,열무나물,90.0,73.8,75.33,3.52,4.77,4.28,0.54,3.60,124.2,...,2446.2,0.00,0.09,0.00,3.14,0.0,0.00,0.59,0.16,81.0
3,열무나물_된장,90.0,61.2,77.49,3.34,3.68,3.68,0.66,3.60,115.2,...,1512.0,0.00,0.18,0.00,0.30,0.0,0.00,0.47,0.11,81.0
4,유채나물,90.0,86.4,69.75,5.22,4.14,7.04,1.78,4.23,144.9,...,2429.1,0.05,0.18,0.09,2.72,0.0,0.00,0.53,0.04,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,샐러드_새싹,100.0,74.0,86.60,2.77,4.81,4.96,2.26,1.50,49.0,...,361.0,0.00,0.11,0.89,1.59,0.0,12.28,0.62,0.02,100.0
487,샐러드_양배추,150.0,225.0,114.30,4.04,17.57,12.71,11.92,3.90,42.0,...,394.5,0.29,0.07,0.47,1.36,0.0,7.71,2.68,0.08,225.0
488,샐러드_양상추,150.0,189.0,120.45,7.36,15.45,5.13,2.37,3.75,66.0,...,294.0,0.76,0.13,1.51,0.98,0.0,0.00,1.96,0.08,225.0
489,샐러드_옥수수,160.0,265.6,114.88,3.14,19.10,20.29,9.62,4.16,24.0,...,403.2,0.09,0.06,0.00,0.59,0.0,9.31,3.01,0.14,256.0


In [105]:
# 함수 실행
adjust_nutrient_values(df_02)

/var/folders/gp/txd8895x3vbc94rnxd7b3gjw0000gn/T/ipykernel_5219/2339764252.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace('-', 0, inplace=True)


,식품명,1회제공량,에너지(kcal),수분(g),단백질(g),지방(g),탄수화물(g),당류(g),식이섬유(g),칼슘(mg),...,베타카로틴(μg),티아민(mg),리보플라빈(mg),니아신(mg),비타민 C(mg),비타민 D(μg),콜레스테롤(mg),포화지방산(g),트랜스지방산(g),1회제공량
0,강냉이,20.0,90.00,0.6,1.00,12,81,20,0.4,2,...,0,0,0.389,1.99,0,0,4.17,4.67,0.42,4.0
1,캐러멜팝콘,20.0,91.80,2.4,0.95,14.36,77.74,34.78,8.6,22,...,39,0.258,0.015,2.193,0.44,0,22.12,8.01,0.32,4.0
2,튀밥,20.0,80.00,4.9,4.15,2.3076923076923100,84.61538461538460,7.6923076923076900,2,21,...,0,0,0.034,0.117,0,0,0,0.7692307692307690,0,4.0
3,팝콘,20.0,92.00,1.9,0.80,16,76,36,10.9,6,...,32,0.011,-,-,0,0,28,8,0,4.0
4,도넛/완제품,70.0,326.67,31.5,4.12,27.45098039215690,49.01960784313730,15.686274509803900,-,27,...,0,0.1,0.07,0.5,0,-,0,13.72549019607840,0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,감자수프,150.0,642.54,0,10.97,10.28460602545390,76.63609229267190,23.377098031351300,3.8004000000000000,35.38845666164430,...,68.27455966343420,0.024011529372448900,0.0578370236658323,0.7262415495651990,0,-,15.628832063077600,5.471051504614150,0.22528844867836900,225.0
568,브로콜리수프,150.0,600.00,82.34702,10.50,4.67,82,48,0.7540786153288340,48.712599504948200,...,31.313651983285400,0.04300659751794130,0.04755689189087780,0.1696316217951950,1.163522933894180,-,13.33,4.27,0,225.0
569,옥수수수프,150.0,592.50,4.470398865297570,7.50,8.5,75,25,4.910471660475820,388.5843646232090,...,57.64926738060430,24.540766666666700,0.4473300222277430,1.3689727021836700,6.759900353940020,-,0,4.5,0,225.0
570,크림수프/액상,150.0,652.50,82.8,7.50,15,70,25,0,14,...,28,0.013,0.046,0.392,0.1,0,25,9,0,225.0


In [115]:
# # 전처리 csv 파일 저장
# df_01.to_csv(file_path + '2단계 가공 데이터/01_filtered_data_공공데이터.csv', index=False, encoding='utf-8-sig')
# df_02.to_csv(file_path + '2단계 가공 데이터/02_filtered_data_식품의약처_가공.csv', index=False, encoding='utf-8-sig')

In [116]:
# 03_Dataset null을 "-"로 되어있어 수정 후 제거 필요
# 전처리 필요 : 1회제공량에 내용량_단위를 붙이기

# 03_Dataset 추가적인 전처리 과정 함수
def addtional_03_dataset(df):
    
    # 03_Dataset은 null값을 '-'로 되어있어 NaN으로 변경
    df.replace('-', pd.NaT, inplace=True)

    # '-' 값 제거
    df = df.dropna(subset=['1회제공량', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '당류(g)', '식이섬유(g)', '나트륨(mg)'], how='any')

    # '1회제공량' 속성에 '내용량_단위' 값을 붙여 출력
    df['1회제공량'] = df.apply(lambda x: f"{x['1회제공량']}{x['내용량_단위']}", axis=1)

    # '내용량_단위' 속성 제거
    df.drop(columns=['내용량_단위'], inplace=True)
    
    return df

In [117]:
# 전처리 필요한 파일 업로드
df_03 = pd.read_csv(file_path + '1단계 가공 데이터/03_filtered_data_식품영양성분_음식.csv')

In [118]:
# 함수 실행
df_03 = addtional_03_dataset(df_03)

/var/folders/gp/txd8895x3vbc94rnxd7b3gjw0000gn/T/ipykernel_5219/3750808802.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1회제공량'] = df.apply(lambda x: f"{x['1회제공량']}{x['내용량_단위']}", axis=1)
/var/folders/gp/txd8895x3vbc94rnxd7b3gjw0000gn/T/ipykernel_5219/3750808802.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['내용량_단위'], inplace=True)


In [119]:
# # 전처리 csv 파일 저장
# df_03.to_csv(file_path + '2단계 가공 데이터/03_filtered_data_식품영양성분_음식.csv', index=False, encoding='utf-8-sig')

### 03.필요 속성을 제외한 제거 진행
1. 식품명
2. 1회제공량
3. 에너지(kcal)
4. 탄수화물(g)
5. 단백질(g)
6. 지방(g)
7. 당류(g)
8. 식이섬유(g)
9. 나트륨(mg)

In [123]:
# 필요속성 제외 제거 함수 정의
def select_necessary_columns(df):
    
    # 필요한 속성만 포함
    necessary_columns = [
        '식품명', '1회제공량', '에너지(kcal)', '탄수화물(g)', 
        '단백질(g)', '지방(g)', '당류(g)', '식이섬유(g)', '나트륨(mg)'
    ]

    # 데이터셋 재정의
    df = df[necessary_columns]
    
    return df

In [124]:
# 파일 업로드
df_01 = pd.read_csv(file_path + '2단계 가공 데이터/01_filtered_data_공공데이터.csv')
df_02 = pd.read_csv(file_path + '2단계 가공 데이터/02_filtered_data_식품의약처_가공.csv')
df_03 = pd.read_csv(file_path + '2단계 가공 데이터/03_filtered_data_식품영양성분_음식.csv')

In [125]:
# 함수 실행
df_01 = select_necessary_columns(df_01)

In [126]:
# 함수 실행
df_02 = select_necessary_columns(df_02)

In [127]:
# 함수 실행
df_03 = select_necessary_columns(df_03)

In [128]:
# # 전처리 csv 파일 저장
# df_01.to_csv(file_path + '3단계 가공 데이터/01_filtered_data_식품영양성분_음식.csv', index=False, encoding='utf-8-sig')
# df_02.to_csv(file_path + '3단계 가공 데이터/02_filtered_data_식품의약처_가공.csv', index=False, encoding='utf-8-sig')
# df_03.to_csv(file_path + '3단계 가공 데이터/03_filtered_data_식품영양성분_음식.csv', index=False, encoding='utf-8-sig')

### 04.데이터셋끼리 merge 및 동일한 식품명이 존재한다면 특정 데이터셋 기준으로 하여 나머지 데이터는 제외
- 03_Dataset 기준 선정 : 1회제공량 속성이 기존에 존재

In [137]:
# 3개의 데이터셋 병합
def merge_datasets(df1, df2):
    # df1을 기준으로 df2를 병합
    merged_df = pd.merge(df1, df2, on='식품명', how='left', suffixes=('', '_df2'))
    
    # 중복되는 식품명의 행에서 df1 데이터를 유지하고 df2 데이터는 삭제
    for col in df1.columns:
        if col != '식품명':
            merged_df[col] = merged_df.apply(lambda x: x[col] if pd.notnull(x[col]) else x[f"{col}_df2"], axis=1)
            merged_df.drop(columns=[f"{col}_df2"], inplace=True)
    
    return merged_df

In [138]:
# 파일 업로드
df_01 = pd.read_csv(file_path + '3단계 가공 데이터/01_filtered_data_공공데이터.csv')
df_02 = pd.read_csv(file_path + '3단계 가공 데이터/02_filtered_data_식품의약처_가공.csv')
df_03 = pd.read_csv(file_path + '3단계 가공 데이터/03_filtered_data_식품영양성분_음식.csv')

In [139]:
# 함수 실행
merged_df1_df2 = merge_datasets(df_03, df_01)

In [140]:
# 함수 실행 
final_merge = merge_datasets(merged_df1_df2, df_02)

In [143]:
# final_merge 데이터셋의 '식품명' 내림차순 정렬
final_merge = final_merge.sort_values(by='식품명')

In [144]:
# 전처리 csv 파일 저장
final_merge.to_csv(file_path + '4단계 가공 데이터/food_data.csv', index=False, encoding='utf-8-sig')

### 05. Excel을 이용하여 2차 중복제거 수행
- 동일한 식품이지만 “식품명”의 텍스트가 띄워쓰기와 같이 조금씩 다를 경우 중복 제거가 되지 않을 경우가 존재하기 때문에 수작업으로 제거 

In [145]:
final_df = pd.read_csv(file_path + '5단계 가공 데이터/food_data.csv')

### 06.음식 분류 모델의 라벨링 데이터와 음식명 맞추기